In [ ]:
!pip install flash-attn --upgrade

In [ ]:
import torch
from flash_attn import flash_attn_func

In [3]:
import torch
batch_size=32
seq_len=2048
hidden_dim=1024


Q = torch.randn(batch_size, seq_len, hidden_dim, dtype=torch.float16)
K = torch.randn(batch_size, seq_len, hidden_dim, dtype=torch.float16)
V = torch.randn(batch_size, seq_len, hidden_dim, dtype=torch.float16)


In [4]:
Q

tensor([[[-8.8916e-01, -7.9883e-01,  7.8467e-01,  ...,  6.1279e-01,
          -1.1221e+00, -8.4277e-01],
         [-1.2146e-01,  1.9736e+00,  2.7100e-01,  ..., -1.3896e+00,
          -1.4270e-01,  7.4609e-01],
         [-8.6084e-01, -5.4980e-01, -1.6055e+00,  ..., -6.3721e-01,
           1.1953e+00, -1.2170e-01],
         ...,
         [-3.0664e-01, -5.7324e-01, -1.2581e-02,  ...,  5.5273e-01,
           1.6484e+00,  5.4053e-01],
         [ 1.1230e+00, -6.3184e-01, -4.1309e-01,  ...,  5.5322e-01,
          -1.2578e+00, -5.3271e-01],
         [-8.2568e-01, -3.1543e-01,  7.7539e-01,  ..., -1.4512e+00,
           1.3730e+00, -7.9297e-01]],

        [[ 1.4746e+00,  4.0527e-01, -1.3779e+00,  ...,  2.5610e-01,
          -1.0430e+00,  8.1689e-01],
         [-9.9060e-02,  1.0852e-01,  1.0586e+00,  ...,  1.0146e+00,
           9.1699e-01,  6.9189e-01],
         [-1.0537e+00,  7.6782e-02, -4.7827e-01,  ..., -1.8982e-01,
          -3.8916e-01, -7.2266e-01],
         ...,
         [-3.1708e-02, -1

In [9]:
import torch.nn.functional as F
import math

## attention
def attention(q, k, v, mask=None, dropout=None):
    d_k = q.size(-1)
    scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim=-1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, v), p_attn

RuntimeError: The size of tensor a (32) must match the size of tensor b (1024) at non-singleton dimension 0